In [22]:
%matplotlib tk
import warnings
warnings.filterwarnings('once') #only get the hyperspy load warning once

import pycrystem as pc
import numpy as np
import hyperspy.api as hs
from matplotlib import pyplot as plt

In [23]:
from pycrystem.indexation_generator import *
from pycrystem.utils.expt_utils import *
from pycrystem import ElectronDiffraction
from pycrystem.indexation_generator import IndexationGenerator,mapable_radial_matcher

In [46]:
from load_data import dp #see file for sizing etc
dp#.inav[0,6].plot()

In [25]:
from generate_library import structure,rot_list,edc,diff_gen,struc_lib
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.,
                                            representation='euler')

 97%|█████████▋| 29492/30351 [01:06<00:02, 309.82it/s]

In [26]:
indexer = IndexationGenerator(dp, library)
match_results = indexer.correlate(n_largest=5)
## Does this actually give correct answers? Potentially not ... great...

A Jupyter Widget

In [29]:
match_results.plot()

In [48]:
#dp.inav[0,0].plot()
best_fit = match_results.inav[0,6].data[0][1:4] 
best_fit_pattern = library['CsPbBr3'][(best_fit[0],best_fit[1],best_fit[2])]

In [49]:
best_fit_pattern.as_signal(144,0.03,1.2).plot()

<h1> PART II - Speeding up the process using radial forms of the data</h1>

In [8]:
radial_averaged_data = dp.map(radial_average,center=np.asarray([144/2,144/2]),cython=False,inplace=False, show_progressbar=False)
## is this using the correct center

In [9]:
### This is being rewritten. But should do via Silicon
radial_sims = {}
for XZX in library['CsPbBr3']:
    peaks_2D_data = library['CsPbBr3'][XZX].as_signal(144,0.3,2.1).data
    radial_sims[XZX] = radial_average(peaks_2D_data,center=[144/2,144/2],cython=False)

CPU times: user 1min 27s, sys: 1.78 s, total: 1min 29s
Wall time: 59.2 s


In [10]:
%%time
alpha = radial_averaged_data.map(mapable_radial_matcher,dict_of_radial_patterns=radial_sims,n_scorer=10,inplace=False)

A Jupyter Widget